In [4]:
genre_map = {}
genre_map['10759'] = 'Action & Adventure'
genre_map['16'] = 'Animation'
genre_map['35'] = 'Comedy'
genre_map['80'] = 'Crime'
genre_map['99'] = 'Documentary'
genre_map['18'] = 'Drama'
genre_map['10751'] = 'Family'
genre_map['10762'] = 'Kids'
genre_map['9648'] = 'Mystery'
genre_map['10763'] = 'News'
genre_map['10764'] = 'Reality'
genre_map['10765'] = 'Sci-Fi & Fantasy'
genre_map['10766'] = 'Soap'
genre_map['10767'] = 'Talk'
genre_map['10768'] = 'War & Politics'
genre_map['37'] = 'Western'

n = '10768'
genre_map[n]

'War & Politics'

In [5]:
import pandas as pd

df = pd.read_json("data/drama_list.json")

# convert json to csv
df.to_csv("data/drama_list.csv", index=False)


In [6]:
df

,adult,backdrop_path,genre_ids,id,origin_country,original_language,original_name,overview,popularity,poster_path,first_air_date,name,vote_average,vote_count
0,False,/5yk3QwnQxzwKfhU1tlDkV2PSIBz.jpg,[18],219246,[KR],ko,폭싹 속았수다,"In Jeju, a spirited girl and a steadfast boy's...",112.8772,/Afrz3QAcQcsT6w4S5wrKUxmYPWM.jpg,2025-03-07,When Life Gives You Tangerines,9.003,162
1,False,/2meX1nMdScFOoV4370rqHWKmXhY.jpg,"[10759, 9648, 18]",93405,[KR],ko,오징어 게임,Hundreds of cash-strapped players accept a str...,75.5149,/1QdXdRYfktUSONkl1oD5gc6Be0s.jpg,2021-09-17,Squid Game,7.900,15677
2,False,/a8a8z4aJr40zYgRMfmSESppX2eO.jpg,"[10764, 35]",70672,[KR],ko,아는 형님,Male celebs play make-believe as high schooler...,58.0235,/2jIi55JtYKJTL1km8qHMuUilOWo.jpg,2015-12-05,Men on a Mission,7.400,33
3,False,/hQ4GRoPuF3f6whlclxveFoOODnk.jpg,"[18, 9648, 10765]",135340,[KR],ko,해피니스,A new type of deadly virus spread throughout t...,72.1660,/j3Q4Vamcb0p7uoXMkkvIp36oMJe.jpg,2021-11-05,Happiness,8.100,499
4,False,/uBp9d5J58yzjafKiWsz7cdctmIY.jpg,[10764],73036,[KR],ko,냉장고를 부탁해,The best chefs of Korea compete to create impr...,52.1553,/zc7qvd7CVuYpWMXZxuSbhG8n8g1.jpg,2014-11-17,Please Take Care of My Refrigerator,8.500,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,False,/8qxwlLR8ZZPXhPIgtWkSPch2bnS.jpg,[],273486,[CN],zh,最强战士迷你特工队之英雄的诞生,,0.1307,/qhmUpsZqEHxoJF3CUnK4dVNimPP.jpg,,最强战士迷你特工队之英雄的诞生,0.000,0
392,False,None,[],273072,[CN],zh,下一站幸福,,0.1039,/ewgxYFK1lwPXRe1ko4mbPqklhC5.jpg,2024-04-22,下一站幸福,0.000,0
393,False,/qsH2Vdgj3eSYHTnWiWr2jJNT1Re.jpg,[],282833,[JP],zh,ドラゴン桜2,,0.0962,/70BwsELorgoGdX8mDo0YsR2gbLP.jpg,,ドラゴン桜2,0.000,0
394,False,/v1bihui37fU9ThGSHUgOh2pEmin.jpg,[99],132048,[CN],zh,我粉你,"In this five-part documentary series, take a d...",0.0618,/1vSAcmMLPcXL6t6R2prnZnv7i8i.jpg,2021-08-18,My Wiggly Friend,5.500,2


In [7]:
# Check for setting up chromadb connection
import psutil
print(f"Memory usage before: {psutil.virtual_memory().percent}%")

Memory usage before: 66.6%


In [20]:
import chromadb
from chromadb.utils import embedding_functions

# setup embedding
default_ef = embedding_functions.DefaultEmbeddingFunction()

# set up chromadb connection
# client = chromadb.EphemeralClient()
client = chromadb.PersistentClient()

# client.delete_collection('collection')

collection = client.get_or_create_collection(
    name='collection',
    embedding_function=default_ef
    )

In [11]:
def add_to_db(info, show_id):
    collection.add(
        documents=[info],                    # store generated info as document
        metadatas=[{'show_id': show_id}],   # store show id as metadata
        ids=[f'{collection.count() + 1}']
    )

In [12]:
import csv

with open("data/drama_list.csv") as file:
    lines = csv.reader(file)

    for i, line in enumerate(lines):
        # skip first row (column titles)
        if i == 0:
            continue

        # generate document to be stored in ChromaDB
        document = f"The tv show name is: {line[11]}\n"
        
        country = line[5]
        if country == "ko":
            document += f"The show classifies as: korean drama \n"
        else:
            document += f"The show classifies as: chinese drama \n"

        genres = ""
        genre_ids = line[2].replace("[", "").replace("]", "")
        if len(genre_ids) > 0:
            for g in genre_ids.split(", "):
                genres += f"{genre_map[g]}, "
        
        document += f"The genre is: {genres}\nThe air date is: {line[10]}\n"
        document += f"The show is about: {line[7]}\nThe popularity is: {line[8]}"

        # store document in ChromaDB
        add_to_db(document, line[3])

In [13]:
collection.count()

396

In [14]:
def chromadb_user_query(question):
    results = collection.query(
        query_texts=[question], # Chroma will embed this for you
        n_results=5             # how many results to return
    )
    return results

In [15]:
# q = "what are some popular chinese dramas?"
q = "I am looking for comedy romance dramas that will make my heart flutter."
top_matches = chromadb_user_query(q)
print(top_matches['documents'])

[['The tv show name is: Melo Movie\nThe show classifies as: korean drama \nThe genre is: Comedy, Drama, \nThe air date is: 2025-02-14\nThe show is about: A movie buff falls for an aspiring director, igniting a romance that fizzles out too soon. When their paths cross again, can love find its way back?\nThe popularity is: 7.5896', "The tv show name is: Romance Is a Bonus Book\nThe show classifies as: korean drama \nThe genre is: Comedy, Drama, \nThe air date is: 2019-01-26\nThe show is about: A gifted writer who's the youngest editor-in-chief ever at his publishing company gets enmeshed in the life of a former copywriter desperate for a job.\nThe popularity is: 6.0147", 'The tv show name is: A Love So Beautiful\nThe show classifies as: chinese drama \nThe genre is: Comedy, \nThe air date is: 2017-11-09\nThe show is about: The ups and downs of school, family and growing up test the affection between a budding artist and her handsome but indifferent classmate and neighbor.\nThe popularity

In [45]:
res = top_matches['metadatas'][0][1]
res["show_id"]

'85220'

In [19]:
example_result = open("data/example.txt", "r")
m = example_result.read()
m

'[\'The tv show name is: Melo Movie\\nThe show classifies as: korean drama \\nThe genre is: Comedy, Drama, \\nThe air date is: 2025-02-14\\nThe show is about: A movie buff falls for an aspiring director, igniting a romance that fizzles out too soon. When their paths cross again, can love find its way back?\\nThe popularity is: 7.5896\', "The tv show name is: Romance Is a Bonus Book\\nThe show classifies as: korean drama \\nThe genre is: Comedy, Drama, \\nThe air date is: 2019-01-26\\nThe show is about: A gifted writer who\'s the youngest editor-in-chief ever at his publishing company gets enmeshed in the life of a former copywriter desperate for a job.\\nThe popularity is: 6.0147", \'The tv show name is: A Love So Beautiful\\nThe show classifies as: chinese drama \\nThe genre is: Comedy, \\nThe air date is: 2017-11-09\\nThe show is about: The ups and downs of school, family and growing up test the affection between a budding artist and her handsome but indifferent classmate and neighbo

In [55]:
temp = [{'show_id': '219246', 'synopsis': 'Summary: "When Life Gives You Tangerines" follows the story of a spirited girl and a steadfast boy on Jeju Island as they navigate life\'s challenges and joys, showcasing the enduring power of love over time.\n\nRecommendation: Fans of heartwarming K-dramas with a mix of romance and drama will enjoy "When Life Gives You Tangerines." Set against the beautiful backdrop of Jeju Island, this series is a must-watch for those who appreciate stories of love, resilience, and personal growth. Get ready to be swept away by the touching tale of two individuals facing life\'s obstacles together.'}, {'show_id': '73036', 'synopsis': "Title: Please Take Care of My Refrigerator\n\nSummary: Watch as Korea's top chefs battle it out to create delicious dishes using only the ingredients they find in celebrities' refrigerators. \n\nRecommendation: If you enjoy reality shows with a culinary twist and love seeing creativity in action, this show is perfect for you. Get ready to be amazed by the unique dishes these chefs come up with!"}, {'show_id': '135340', 'synopsis': 'Title: Happiness\n\nSummary: In the midst of a deadly virus outbreak, residents from different social classes are forced to coexist in a sealed-off apartment complex. As tensions rise and fear spreads, they must navigate survival and societal conflicts.\n\nRecommendation: Fans of suspenseful dramas with a touch of mystery and sci-fi elements will be drawn to "Happiness." Get ready for a thrilling ride as you watch the residents struggle to survive and overcome their differences in this gripping K-drama.'}, {'show_id': '233686', 'synopsis': 'Summary:\n"Get ready for a gripping tale of karma and crime in the Korean drama \'Karma.\' Six lives are forever changed after a fateful accident, leading them to confront their past and face their darkest truths."\n\nRecommendation:\n"Fans of crime dramas and thrilling mysteries will be captivated by the intertwining storylines and suspenseful twists in \'Karma.\' Make sure to mark your calendars for the premiere on April 4th, 2025!"'}, {'show_id': '70672', 'synopsis': "Summary: Men on a Mission is a hilarious reality show where male celebrities pretend to be high school students and compete in funny challenges each week. With witty humor and silly antics, it's perfect for fans of comedy and reality TV.\n\nRecommendation: If you enjoy light-hearted humor and watching celebrities let loose, Men on a Mission is the perfect show for you. Get ready to laugh out loud as these stars engage in playful competitions and showcase their comedic talents. Grab some popcorn and get ready for a fun and entertaining ride!"}]
len(temp)

# for id, syn in temp[0]:
#     print(f'id: {id}, syn: {syn}')

5